In [ ]:
# Importing required functions for data manipulation
from pyspark.sql.functions import col, count, udf, when, sum, round
from pyspark.sql.types import StringType

In [ ]:
matches_df=spark.read.csv("/FileStore/tables/matches.csv",header='true', inferSchema='true')
deliveries_df=spark.read.csv("/FileStore/tables/deliveries.csv",header='true', inferSchema='true')
auction_df=spark.read.csv("/FileStore/tables/ipl_2025_auction_players.csv",header='true', inferSchema='true')
     
     
     

In [ ]:
# Group by batter and compute batting statistics
batsman_stats_df = deliveries_df.groupBy("batter").agg(
    count("*").alias("balls_faced"),  # Count the number of balls faced
    sum(col("batsman_runs")).alias("total_runs"),  # Sum the total runs scored
    sum((col("batsman_runs") == 4).cast("int")).alias("fours"),  # Count the number of fours
    sum((col("batsman_runs") == 6).cast("int")).alias("sixes")  # Count the number of sixes
).withColumn(
    "strike_rate", round((col("total_runs") * 100 / col("balls_faced")), 2)  # Calculate strike rate
).withColumn(
    "boundaries", col("fours") + col("sixes")  # Calculate total boundaries (fours + sixes)
).filter(
    col("balls_faced") > 50  # Filter out batters who faced more than 50 balls
).orderBy("strike_rate", ascending=False)  # Order by strike rate in descending order

In [ ]:
# Filter out valid wickets (excluding run outs)
valid_wickets_df = deliveries_df.filter(
    (col("is_wicket") == 1) & (col("dismissal_kind") != "run out")
)

# Group by bowler and count the number of wickets
bowler_wickets_df = valid_wickets_df.groupBy("bowler").agg(count("*").alias("wickets")).orderBy(col("wickets").desc())
bowler_wickets_df = bowler_wickets_df.withColumn(
    "bowler",
    when(col("bowler") == "A Singh", "Amit Singh").otherwise(col("bowler"))
)

In [ ]:
# Filter out valid wickets (excluding run outs)
valid_wickets_df = deliveries_df.filter(
    (col("is_wicket") == 1) & (col("dismissal_kind") != "run out")
)

# Compute total runs conceded per bowler
bowler_stats_df = deliveries_df.groupBy("bowler").agg(
    count(when((col("is_wicket") == 1) & (col("dismissal_kind") != "run out"), 1)).alias("wickets"),
    sum(col("total_runs")).alias("runs_conceded"),
    count(when(col("ball") == 6, 1)).alias("overs_bowled")  # Count number of completed overs
)

# Calculate Economy Rate: runs conceded per over
bowler_stats_df = bowler_stats_df.withColumn(
    "economy_rate", round(col("runs_conceded") / (col("overs_bowled") + 0.1), 2)  # Avoid division by zero
).withColumn(
    "bowler", when(col("bowler") == "A Singh", "Amit Singh").otherwise(col("bowler"))
).orderBy(col("wickets").desc())



In [ ]:
# UDF to check if the first name is all uppercase
def is_all_caps(name):
    return name.isupper()  # Check if it's all uppercase

# Define UDF to format names correctly
def format_name(full_name):
    parts = full_name.split()
    if is_all_caps(parts[0]):
        return f"{parts[0][0]} {parts[-1]}"  # Keep only first initial + last name
    return full_name  # If there's only one word, return as is

format_name_udf = udf(format_name, StringType())

# Apply the transformation
batsman_stats_final_df = batsman_stats_df.withColumn("player", format_name_udf(col("batter")))

bowler_wickets_final_df = bowler_stats_df.withColumn("player", format_name_udf(col("bowler")))


In [ ]:
unsold_players = auction_df.filter(col("Sold") == "Unsold").dropDuplicates()

In [ ]:
# Define a UDF to convert full names to initial-based names
def convert_to_initials(full_name):
    parts = full_name.split()
    if len(parts) > 1:
        return f"{parts[0][0]} {parts[1]}"  # e.g., "Travis Head" → "TM Head"
    return full_name  # If there's only one name, keep it unchanged

convert_to_initials_udf = udf(convert_to_initials, StringType())

# Apply transformation to create a new column with formatted names
unsold_players_df = unsold_players.withColumn("player", convert_to_initials_udf(col("Players")))

In [ ]:
# Join the unsold players with the batsman stats on the transformed name
unsold_batsmen_stats_df = unsold_players_df.join(
    batsman_stats_final_df, on="player"
).select("Players", "total_runs", "balls_faced", "strike_rate", "boundaries")

# Show the final output
display(unsold_batsmen_stats_df.orderBy("strike_rate", ascending=False).limit(10))


In [ ]:
Players	     total_runs	  balls_faced	strike_rate	  boundaries
Shai Hope	    183	        125	        146.4	        24
Prithvi Shaw	1892	    1321	    143.22	        299
Jonny Bairstow	1589	    1132	    140.37	        235
Kyle Mayers	    379	        276	        137.32	        60
David Warner	6567	    4849	    135.43	        899
Shardul Thakur	307	        228	        134.65	        40
Dewald Brevis	230	        176	        130.68	        33
Sarfaraz Khan	585	        456	        128.29	        77
Daryl Mitchell	351	        274	        128.1	        38
Steven Smith	2495	    1999	    124.81	        286


In [ ]:
display(unsold_batsmen_stats_df.orderBy("boundaries", ascending=False).limit(10))

In [ ]:
Players	        total_runs	    balls_faced	    strike_rate	    boundaries
David Warner	6567	        4849	        135.43	        899
Prithvi Shaw	1892	        1321	        143.22	        299
Steven Smith	2495	        1999	        124.81	        286
Kane Williamson	2132	        1734	        122.95	        251
Jonny Bairstow	1589	        1132	        140.37	        235
Sarfaraz Khan	585	            456	            128.29	        77
Piyush Chawla	624	            580	            107.59	        76
Kyle Mayers	    379	            276	            137.32	        60
Shardul Thakur	307	            228	            134.65	        40
Daryl Mitchell	351	            274	            128.1	        38


In [ ]:
# Join the unsold players with the batsman stats on the transformed name
unsold_bowler_stats_df = unsold_players_df.join(
    bowler_wickets_final_df, on="player"
).select("Players", "bowler", "wickets","economy_rate")

# Show the final output
display(unsold_bowler_stats_df.orderBy("wickets", ascending=False).limit(10))
     

In [ ]:
Players	        total_runs	balls_faced	    strike_rate	    boundaries
David Warner	   6567	    4849	        135.43	        899
Prithvi Shaw	   1892	    1321	        143.22	        299
Steven Smith	   2495	    1999	        124.81	        286
Kane Williamson	   2132	    1734	        122.95	        251
Jonny Bairstow	   1589	    1132	        140.37	        235
Sarfaraz Khan	   585	    456	            128.29	        77
Piyush Chawla	   624	    580	            107.59	        76
Kyle Mayers	        379	    276	            137.32	        60
Shardul Thakur	    307	    228	            134.65	        40
Daryl Mitchell	    351	    274	            128.1	        38

Databricks visualization. Run in Databricks to view.

In [ ]:
display(unsold_bowler_stats_df.filter("wickets > 10").orderBy("economy_rate").limit(10))
     
Players	        bowler	    wickets	    economy_rate
Piyush Chawla	PP Chawla	192	        8.12
Murugan Ashwin	M Ashwin	35	        8.2
Rishi Dhawan	R Dhawan	25	        8.37
Chetan Sakariya	C Sakariya	20	        8.58
Umesh Yadav	    UT Yadav	144	        8.74
Jason Behrendorff JP Behrendorff	19	9.21
Naveen-ul-Haq	Naveen-ul-Haq	25	    9.39
Shardul Thakur	SN Thakur	94	        9.44
Chris Jordan	CJ Jordan	30	        9.91
Alzarri Joseph	AS Joseph	21	        9.94


Databricks visualization. Run in Databricks to view.